In [ ]:
!pip install python-mnist

In [ ]:
%tensorflow_version 2.x

# use this to work with tensorflow 2.x on colab

In [ ]:
!wget https://www.dropbox.com/s/y54njjo7w117bon/mnist_data.zip

In [ ]:
!unzip mnist_data

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from mnist import MNIST
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

mndata = MNIST(r'/content/mnist_data')

In [ ]:
images_train, labels_train = mndata.load_training()
images_test,labels_test=mndata.load_testing()

In [ ]:
labels_train=np.array(labels_train).reshape(-1,1)
labels_test=np.array(labels_test).reshape(-1,1)

In [ ]:
onehot=OneHotEncoder()

onehot.fit(labels_train)

labels_train=onehot.transform(labels_train)
labels_test=onehot.transform(labels_test)
labels_train=labels_train.toarray()
labels_test=labels_test.toarray()

In [ ]:
images_train=np.array(images_train).reshape(60000,784)/255
images_test=np.array(images_test).reshape(10000,784)/255

In [ ]:
images_train=images_train.reshape(60000,  28, 28,1).astype('float32')
images_test=images_test.reshape(10000, 28, 28,1).astype('float32')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D

In [ ]:
# Sequential API 

In [ ]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=( 28, 28,1), activation='relu',
                 padding='same'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5),activation='relu',padding='same'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(200, activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
# Functional API

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:


image=Input(shape=(28,28,1))

conv1=Conv2D(32, (5, 5), activation='relu',padding='same')(image)
pool1=MaxPooling2D(pool_size=(2, 2))(conv1)

conv2=Conv2D(64, (5, 5),activation='relu',padding='same')(pool1)
pool2=MaxPooling2D(pool_size=(2, 2))(conv2)

flat_layer=Flatten()(pool2)

dense1=Dense(1024, activation='relu')(flat_layer)
drop1=Dropout(0.2)(dense1)

dense2=Dense(200, activation='relu')(drop1)
drop2=Dropout(0.2)(dense2)

output=Dense(10, activation='softmax')(drop2)

model=Model(inputs=image,outputs=output)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
plot_model(model)

In [ ]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
model.fit(images_train,labels_train,validation_data=[images_test,labels_test],
         epochs=1,batch_size=50,verbose=1)

In [ ]:
# Saving the model 

In [ ]:
from tensorflow.keras.models import model_from_json

In [ ]:
model_json = model.to_json()

In [ ]:
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights("model.h5")

In [ ]:
# loading the model 

In [ ]:
from tensorflow.keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
loaded_model.summary()

In [ ]:
loaded_model.load_weights("model.h5")

In [ ]:
# make prediction 

In [ ]:
p=loaded_model.predict(images_test)

In [ ]:
labels_test_predicted=np.argmax(p,1)

In [ ]:
# first argument to pd.crosstab goes to rows, second to columns
# row_0 : real labels
# col_0 : predicted labels
pd.crosstab(np.argmax(labels_test,axis=1),labels_test_predicted)

In [ ]:
y_true=np.argmax(labels_test,axis=1)

In [ ]:
labels_test_predicted

In [ ]:
(y_true==labels_test_predicted).sum()

In [ ]:
a=y_true==labels_test_predicted

In [ ]:
c=[i for i,j in enumerate(a) if not j]

In [ ]:
c

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ind=7434
sample_image = images_test[ind,:,:,:] 
# sample_image = np.array(sample_image, dtype='float')
pixels = sample_image.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()
print('real label:',np.argmax(labels_test[ind]))
print('predicted label:',labels_test_predicted[ind])

In [ ]:
# you can make prediction directly but if you want to further train from here 
# you'll have to compile the loaded model again 

In [ ]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
loaded_model.fit(images_train,labels_train,validation_data=[images_test,labels_test],
         epochs=3,batch_size=50,verbose=1)